# The Winning Formula: What Makes a Game a Hit?


<div class="alert alert-block alert-info">
<b>
    
- Entrego este proyecto completamente en inglés para incrementar su valor curricular en mi portafolio.
    
- Hago esta primera entrega incompleta para continuar con el siguiente sprint mientras completo este proyecto.
</b> <a class="tocSkip"></a>
</div>

In this project, we will analize data from the online videogame store Ice. We want to identify patterns that determine whether a game is successful or not, and use this knowledge to plan effective marketing campaigns for promising projects.

## Introduction
The raw data is in the file `/datasets/games.csv` and contains records of up to 2016. We will build forecasts for 2017.

#### Data description
The data comes with the following fields (columns):

— Name

— Platform

— Year_of_Release

— Genre

— NA_sales (North America sales in millions of USD) 

— EU_sales (Europe sales in millions of USD) 

— JP_sales (Japan sales in millions of USD) 

— Other_sales (other countries sales in millions of USD) 

— Critic_Score (maximum of 100) 

— User_Score (maximum of 10) 

— Rating (ESRB)

The `Rating` column contains the rating given by the Entertainment Software Rating Board, which evaluates the game content and assigns it an age classification, such as Teen or Adult.

## Data loading

In [1]:
import pandas as pd

In [2]:
# Load the data from file
games = pd.read_csv('/datasets/games.csv')

In [16]:
# Show a sample of the data
games.sample(5)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
706,Wario Land 4,GBA,2001.0,Platform,0.90,0.73,0.54,0.09,88.0,8.9,E
12084,Fantastic Four,PS,1997.0,Action,0.04,0.03,0.00,0.00,NaN,NaN,NaN
2250,NHL 98,PS,1997.0,Sports,0.51,0.35,0.00,0.06,NaN,NaN,NaN
6313,NHL 07,X360,2006.0,Sports,0.25,0.00,0.00,0.02,79.0,5.3,E10+
11063,Go Play: City Sports,Wii,2009.0,Sports,0.08,0.00,0.00,0.01,NaN,tbd,E


In [4]:
# Show the dataframe info
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [34]:
print('Years:', sorted(games['Year_of_Release'].unique()),'\n')
print('Genres:', games['Genre'].unique(),'\n')
print('Ratings:', games['Rating'].unique(),'\n')
print('Critic Scores:', sorted(games['Critic_Score'].unique()),'\n')
print('User Scores:', games['User_Score'].unique(),'\n')

Years: [1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, nan] 

Genres: ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan] 

Ratings: ['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP'] 

Critic Scores: [76.0, nan, 13.0, 17.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 

We observe that the dataset comprises games from every year **between 1980 and 2016**, from **11 different genres** and **8 different ratings**. We also notice that **none** of the `Sales` or the `Platform` columns have empty values.

## Data Cleaning & Preparation

For convenience and good practice we convert the column names to lower case.

In [66]:
games.columns = games.columns.str.lower()

### Data types
The years and the critic scores have only integers (besides null values), hence we can convert them to type `Int64`, which accepts nan values. 
`user_score`, however, cannot be converted to a numeric type, as it also includes the string 'tbd'.

In [68]:
# Convert columns type ton integers
games['year_of_release'] = games['year_of_release'].astype('Int64')
games['critic_score'] = games['critic_score'].astype('Int64')

### Empty values

In [69]:
# Show games with empty name
games[games['name'].isnull()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993,NaN,1.78,0.53,0.00,0.08,<NA>,NaN,NaN
14244,NaN,GEN,1993,NaN,0.00,0.00,0.03,0.00,<NA>,NaN,NaN


In [70]:
games.query("platform == 'GEN' and year_of_release == 1993")

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993,NaN,1.78,0.53,0.00,0.08,<NA>,NaN,NaN
7885,Shining Force II,GEN,1993,Strategy,0.00,0.00,0.19,0.00,<NA>,NaN,NaN
8893,Super Street Fighter II,GEN,1993,Fighting,0.00,0.00,0.15,0.00,<NA>,NaN,NaN
11986,Ecco: The Tides of Time,GEN,1993,Adventure,0.00,0.00,0.07,0.00,<NA>,NaN,NaN
12098,Street Fighter II': Special Champion Edition (...,GEN,1993,Action,0.00,0.00,0.07,0.00,<NA>,NaN,NaN
12264,Streets of Rage 3,GEN,1993,Action,0.00,0.00,0.07,0.00,<NA>,NaN,NaN
12984,Dynamite Headdy,GEN,1993,Platform,0.00,0.00,0.05,0.00,<NA>,NaN,NaN
13343,Beyond Oasis,GEN,1993,Role-Playing,0.00,0.00,0.05,0.00,<NA>,NaN,NaN
14244,NaN,GEN,1993,NaN,0.00,0.00,0.03,0.00,<NA>,NaN,NaN


There are only **2 games without name**. Both from the year 1993 and for the console 'GEN' (_Sega Genesis_), which had more games release on the same year.

!! For the moment we leave them as it is...

### Duplicates

In [71]:
# Show games with duplicated names and console
games[games[['name','platform']].duplicated()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
1591,Need for Speed: Most Wanted,X360,2005,Racing,1.0,0.13,0.02,0.10,83,8.5,T
4127,Sonic the Hedgehog,PS3,<NA>,Platform,0.0,0.48,0.00,0.00,43,4.1,E10+
11715,Need for Speed: Most Wanted,PC,2012,Racing,0.0,0.06,0.00,0.02,82,8.5,T
14244,NaN,GEN,1993,NaN,0.0,0.00,0.03,0.00,<NA>,NaN,NaN
16230,Madden NFL 13,PS3,2012,Sports,0.0,0.01,0.00,0.00,83,5.5,E


In [73]:
# Investigate duplicated entries 
duplicated_names = ['Need for Speed: Most Wanted', 'Sonic the Hedgehog', 'Need for Speed: Most Wan', 'Madden NFL 13']
duplicated_platforms = ['X360', 'PS3', 'PC']

games.query("name in @duplicated_names and platform in @duplicated_platforms").sort_values(by='name')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
507,Madden NFL 13,X360,2012,Sports,2.53,0.15,0.00,0.17,81,5.8,E
604,Madden NFL 13,PS3,2012,Sports,2.11,0.22,0.00,0.23,83,5.5,E
16230,Madden NFL 13,PS3,2012,Sports,0.00,0.01,0.00,0.00,83,5.5,E
523,Need for Speed: Most Wanted,PS3,2012,Racing,0.71,1.46,0.06,0.58,<NA>,NaN,NaN
1190,Need for Speed: Most Wanted,X360,2012,Racing,0.62,0.78,0.01,0.15,83,8.5,T
1591,Need for Speed: Most Wanted,X360,2005,Racing,1.00,0.13,0.02,0.10,83,8.5,T
5972,Need for Speed: Most Wanted,PC,2005,Racing,0.02,0.23,0.00,0.04,82,8.5,T
11715,Need for Speed: Most Wanted,PC,2012,Racing,0.00,0.06,0.00,0.02,82,8.5,T
1745,Sonic the Hedgehog,PS3,2006,Platform,0.41,0.06,0.04,0.66,43,4.1,E10+
1996,Sonic the Hedgehog,X360,2006,Platform,0.44,0.48,0.00,0.11,46,4.4,E10+


These duplicates have different explanations and may need further investigation. However, there are fortunately only a few of them...

## Exploratory Analysis

In [80]:
games.groupby(['year_of_release'])['platform'].count()

year_of_release
1980       9
1981      46
1982      36
1983      17
1984      14
1985      14
1986      21
1987      16
1988      15
1989      17
1990      16
1991      41
1992      43
1993      62
1994     121
1995     219
1996     263
1997     289
1998     379
1999     338
2000     350
2001     482
2002     829
2003     775
2004     762
2005     939
2006    1006
2007    1197
2008    1427
2009    1426
2010    1255
2011    1136
2012     653
2013     544
2014     581
2015     606
2016     502
Name: platform, dtype: int64

## User Profile

## Hypothesis testing


## Conclusions